In [2]:
!pip install transformers

In [3]:
import transformers

In [4]:
!pip install transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.0 MB/s eta 0:00:00


In the previous exercise you saw how sequences get translated into lists of numbers. Let’s convert this list of numbers to a tensor and send it to the model:

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

import torch: Εισάγει τη βιβλιοθήκη PyTorch, η οποία χρησιμοποιείται για την υλοποίηση αλγορίθμων μηχανικής μάθησης, κυρίως σε συνδυασμό με νευρωνικά δίκτυα.

from transformers import AutoTokenizer, AutoModelForSequenceClassification: Εισάγει δύο κλάσεις από το πακέτο Transformers. Η κλάση AutoTokenizer χρησιμοποιείται για τη δημιουργία ενός tokenizer για μοντέλα μεταφοράς (transformer models), ενώ η κλάση AutoModelForSequenceClassification χρησιμοποιείται για τη δημιουργία ενός μοντέλου μεταφοράς για την κατηγοριοποίηση ακολουθιών.

Αν θέλετε να χρησιμοποιήσετε αυτές τις κλάσεις για κάποιο συγκεκριμένο μοντέλο, θα πρέπει να παρέχετε το όνομα του μοντέλου στις κατάλληλες μεθόδους παραδειγμα
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [5]:
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

αρα βάζουμε στο checkpoint το μοντελο και καλουμε το αντιστοιχο tokenizer και το προεκπαιδευμενο μοντελο


In [6]:
sequence = "I've been waiting for a HuggingFace course my whole life."
tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)
input_ids = torch.tensor(ids)

tokens = tokenizer.tokenize(sequence): Χρησιμοποιεί τον tokenizer για να διαχωρίσει την ακολουθία κειμένου σε τοκενς. Το αποτέλεσμα είναι μια λίστα από τοκενς. Για παράδειγμα, η πρόταση "I've been waiting for a HuggingFace course my whole life." μπορεί να διασπαστεί σε τοκενς όπως ["I", "'", "ve", "been", "waiting", "for", "a", "Hugging", "##Face", "course", "my", "whole", "life", "."].

ids = tokenizer.convert_tokens_to_ids(tokens): Χρησιμοποιεί τον tokenizer για να μετατρέψει τα τοκενς σε αντίστοιχα αναγνωριστικά (IDs). Κάθε τοκεν αντιστοιχεί σε ένα μοναδικό αριθμητικό αναγνωριστικό στο λεξικό του μοντέλου. Το αποτέλεσμα είναι μια λίστα από αναγνωριστικά.

input_ids = torch.tensor(ids): Δημιουργεί ένα tensor του PyTorch από τα αναγνωριστικά. Τα tensors είναι δομές δεδομένων που χρησιμοποιούνται συχνά στο πλαίσιο του PyTorch για την αναπαράσταση και τον υπολογισμό στοιχείων στο νευρωνικό δίκτυο.

Συνολικά, ο παραπάνω κώδικας μετατρέπει μια ακολουθία κειμένου σε μια ακολουθία αναγνωριστικών που μπορεί να χρησιμοποιηθεί ως είσοδος σε ένα μοντέλο μηχανικής μάθησης που βασίζεται σε αναγνωριστικά.

In [7]:
model(input_ids)

IndexError: too many indices for tensor of dimension 1

In [8]:
tokenized_inputs = tokenizer(sequence, return_tensors="pt")
print(tokenized_inputs["input_ids"])

tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102]])


The problem is that we sent a single sequence to the model, whereas 🤗 Transformers models expect multiple sentences by default. Here we tried to do everything the tokenizer did behind the scenes when we applied it to a sequence. But if you look closely, you’ll see that the tokenizer didn’t just convert the list of input IDs into a tensor, it added a dimension on top of it

Παμε να το δουμε σωστα


In [10]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)

input_ids = torch.tensor([ids])


μεχρι στιγμησ ειναι ο ιδιος κώδικας τι αλλαζει μετα


In [11]:

print("Input IDs:", input_ids)


Input IDs: tensor([[ 1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,  2607,
          2026,  2878,  2166,  1012]])


In [12]:
output = model(input_ids)
print("Logits:", output.logits)

Logits: tensor([[-2.7276,  2.8789]], grad_fn=<AddmmBackward0>)


We print the input IDs as well as the resulting logits ,
Batching is the act of sending multiple sentences through the model, all at once. If you only have one sentence, you can just build a batch with a single sequence:

In [14]:
batched_ids = [ids, ids]

Η διαδικασία του "batching" επιτρέπει στο μοντέλο να λειτουργήσει όταν του δίνονται πολλές προτάσεις. Η χρήση πολλαπλών ακολουθιών είναι εξίσου απλή με τη δημιουργία ενός "batch" με μια μόνο πρόταση. Υπάρχει, ωστόσο, ένα δεύτερο πρόβλημα. Όταν προσπαθείτε να συνθέσετε δύο (ή περισσότερες) προτάσεις σε ένα "batch", οι προτάσεις αυτές μπορεί να έχουν διαφορετικό μήκος. Αν έχετε εργαστεί με tensors πριν, γνωρίζετε ότι πρέπει να έχουν ορθογώνιο σχήμα (rectangular shape), επομένως δεν θα μπορέσετε να μετατρέψετε απευθείας τη λίστα από αναγνωριστικά εισόδου σε tensor. Για να αντιμετωπίσετε αυτό το πρόβλημα, συνήθως γεμίζουμε τις εισόδους (padding).

PADDING THE INPUTS

In order to work around this, we’ll use padding to make our tensors have a rectangular shape. Padding makes sure all our sentences have the same length by adding a special word called the padding token to the sentences with fewer values. For example, if you have 10 sentences with 10 words and 1 sentence with 20 words, padding will ensure all the sentences have 20 words.

The padding token ID can be found in tokenizer.pad_token_id. Let’s use it and send our two sentences through the model individually and batched together:

In [17]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence1_ids = [[200, 200, 200]]
sequence2_ids = [[200, 200]]
batched_ids = [
    [200, 200, 200],
    [200, 200, tokenizer.pad_token_id],
]



Αυτό το τμήμα κώδικα χρησιμοποιεί το 🤗 Transformers για να φορτώσει ένα προ-εκπαιδευμένο μοντέλο για ταξινόμηση ακολουθιών (sequence classification), και στη συνέχεια παρουσιάζει το μοντέλο με τρεις διαφορετικές εισόδους:

Προσθέτει το sequence1_ids που είναι μια λίστα που περιέχει μια ακολουθία ([200, 200, 200]).
Προσθέτει το sequence2_ids που είναι μια άλλη λίστα που περιέχει μια διαφορετική ακολουθία ([200, 200]).
Προσθέτει το batched_ids που είναι μια λίστα με δύο ακολουθίες. Η πρώτη ακολουθία είναι [200, 200, 200], ενώ η δεύτερη έχει μήκος μικρότερο και χρησιμοποιεί την τιμή tokenizer.pad_token_id για να προσθέσει padding.
Στη συνέχεια, το μοντέλο χρησιμοποιείται για να προβλέψει τις εξόδους (logits) για κάθε μια από αυτές τις εισόδους. Οι εξόδοι προβλέψεων για κάθε περίπτωση εκτυπώνονται με τη χρήση της συνάρτησης print().

Συνοπτικά, η εν λόγω εντολή εκτελεί προβλέψεις για το μοντέλο για τρεις διαφορετικές περιπτώσεις εισόδου και εκτυπώνει τα αποτελέσματα.

print(model(torch.tensor(batched_ids)).logits)
 η εντολή εκτελεί το μοντέλο πάνω στα δεδομένα που περιέχονται στη λίστα batched_ids. Χρησιμοποιεί τη συνάρτηση torch.tensor() για να μετατρέψει τη λίστα σε ένα PyTorch tensor και στη συνέχεια καλεί το μοντέλο πάνω σε αυτό το tensor.

Η έξοδη τιμή της εντολής είναι οι "logits" που παράγονται από το μοντέλο. Οι "logits" είναι αριθμητικές τιμές που προκύπτουν από το μοντέλο πριν από την εφαρμογή ενός συνόλου softmax για την πρόβλεψη πιθανοτήτων κλάσεων. Σε αυτήν την περίπτωση, τα "logits" αναφέρονται σε πιθανότητες για τις κλάσεις της ταξινόμησης των ακολουθιών.

In [18]:
print(model(torch.tensor(sequence1_ids)).logits)
print(model(torch.tensor(sequence2_ids)).logits)
print(model(torch.tensor(batched_ids)).logits)

tensor([[ 1.5694, -1.3895]], grad_fn=<AddmmBackward0>)
tensor([[ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>)
tensor([[ 1.5694, -1.3895],
        [ 1.3374, -1.2163]], grad_fn=<AddmmBackward0>)


There’s something wrong with the logits in our batched predictions: the second row should be the same as the logits for the second sentence, but we’ve got completely different values!

This is because the key feature of Transformer models is attention layers that contextualize each token. These will take into account the padding tokens since they attend to all of the tokens of a sequence. To get the same result when passing individual sentences of different lengths through the model or when passing a batch with the same sentences and padding applied, we need to tell those attention layers to ignore the padding tokens. This is done by using an attention mask.

ATTENTION MASKS


Attention masks are tensors with the exact same shape as the input IDs tensor, filled with 0s and 1s: 1s indicate the corresponding tokens should be attended to, and 0s indicate the corresponding tokens should not be attended to (i.e., they should be ignored by the attention layers of the model).

In [19]:
batched_ids = [
    [200, 200, 200],
    [200, 200, tokenizer.pad_token_id],
]

attention_mask = [
    [1, 1, 1],
    [1, 1, 0],
]

outputs = model(torch.tensor(batched_ids), attention_mask=torch.tensor(attention_mask))
print(outputs.logits)

tensor([[ 1.5694, -1.3895],
        [ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>)


Now we get the same logits for the second sentence in the batch.
Notice how the last value of the second sequence is a padding ID, which is a 0 value in the attention mask.

Longer sequences
With Transformer models, there is a limit to the lengths of the sequences we can pass the models. Most models handle sequences of up to 512 or 1024 tokens, and will crash when asked to process longer sequences. There are two solutions to this problem:

Use a model with a longer supported sequence length.
Truncate your sequences.
Models have different supported sequence lengths, and some specialize in handling very long sequences. Longformer is one example, and another is LED. If you’re working on a task that requires very long sequences, we recommend you take a look at those models.

Otherwise, we recommend you truncate your sequences by specifying the max_sequence_length parameter:

Copied
sequence = sequence[:max_sequence_length]
